In [3]:
import sqlite3
import warnings
import os

import pandas as pd

In [4]:
# Создать соединение с базой данных
connection = sqlite3.connect('../базаданных.sqlite')
connection

pd.io.sql.read_sql('''PRAGMA table_info(pageviews)''', con = connection)


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


In [5]:
pd.io.sql.read_sql('''SELECT *
                      FROM   pageviews
                      LIMIT  10''', con = connection)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


In [6]:
pageviews = pd.io.sql.read_sql('''SELECT uid,
                                         datetime
                                  FROM   pageviews
                                  WHERE  uid LIKE "user_%"
                                  ORDER  BY uid''', con = connection, index_col = 'datetime', parse_dates = ['datetime'])
pageviews

,uid
datetime,
2020-04-26 21:53:59.624136,user_1
2020-04-26 22:06:19.478143,user_1
2020-04-26 22:12:09.614497,user_1
2020-04-30 19:29:01.831635,user_1
2020-05-05 20:26:32.894852,user_1
...,...
2020-04-29 16:51:21.877630,user_30
2020-05-09 20:30:47.034282,user_30
2020-05-22 11:30:18.368990,user_5


In [7]:
pd.io.sql.read_sql('''PRAGMA table_info(checker)''', con = connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [8]:
pd.io.sql.read_sql('''SELECT * 
                      FROM   checker 
                      LIMIT  10''', con = connection)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [9]:
checkers = pd.io.sql.read_sql('''
SELECT Count(uid) AS cnt
FROM   pageviews
WHERE  uid IN (SELECT DISTINCT uid
               FROM   checker AS c
               WHERE  c.status = "ready"
                      AND c.numtrials = 1
                      AND c.labname IN ( "laba04", "laba04s", "laba05", "laba06",
                                         "laba06s", "project1" )) ; ''',
                              con = connection)

checkers

,cnt
0,985


In [10]:
# Объединение данных таблиц pageviews и checker по требованиям
datamart = pd.io.sql.read_sql('''
SELECT c.uid,
       c.labname,
       c.timestamp     AS first_commit_ts,
       Min(p.datetime) AS first_view_ts
FROM   checker AS c
       LEFT JOIN pageviews AS p
              ON c.uid = p.uid
WHERE  c.status = "ready"
       AND c.numtrials = 1
       AND c.labname IN ( "laba04", "laba04s", "laba05", "laba06",
                          "laba06s", "project1" )
       AND c.uid LIKE "user_%"
GROUP  BY c.timestamp,
          c.uid; ''', con = connection, parse_dates = ['first_commit_ts', 'first_view_ts'])

# Создание новой таблицы datamart в базе данных
datamart.to_sql("datamart", con = connection, if_exists="replace")

# Что записали в базу данных?
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [11]:
# Создание датафрейма test
# test = pd.io.sql.read_sql('''SELECT * FROM datamart WHERE first_view_ts IS NOT NULL; ''', con = sql_con_3)

test = datamart[datamart['first_view_ts'].notnull()].copy()

# Запись датафрейма test в базу данных
test.to_sql("test", con = connection, if_exists="replace")
print("Далее отображена информация по датафрейму test:")
test.info()

# Создание датафрейма control
# control = pd.io.sql.read_sql('''SELECT * FROM datamart WHERE first_view_ts IS NULL; ''', con = sql_con_3) 

control = datamart[datamart['first_view_ts'].isnull()].copy()
control.first_view_ts = test.first_view_ts.mean()

# Запись датафрейма control в базу данных
control.to_sql("control", con = connection, if_exists="replace")
print("Далее отображена информация по датафрейму control:")
control.info()

Далее отображена информация по датафрейму test:
<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB
Далее отображена информация по датафрейму control:
<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: da

In [12]:
pd.io.sql.read_sql('''PRAGMA table_info(test)''', con = connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [13]:
pd.io.sql.read_sql('''SELECT * 
                      FROM   test 
                      LIMIT  10''', con = connection)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [14]:
# Найти значения периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания

df_min = pd.io.sql.read_sql('''
SELECT t.uid,
       Cast(Min(Julianday(d.deadlines, 'unixepoch') -
            Julianday(t.first_commit_ts)) * 24 AS
            INTEGER) AS delta
FROM   test AS t
       LEFT JOIN deadlines AS d
              ON t.labname = d.labs
WHERE  t.labname NOT IN ( "project1" ) ; ''', con = connection)

df_min

,uid,delta
0,user_25,2


In [15]:
df_max = pd.io.sql.read_sql('''
SELECT t.uid,
       Cast(Min(Julianday(d.deadlines, 'unixepoch') -
            Julianday(t.first_commit_ts)) * 24 AS
            INTEGER) AS delta
FROM   test AS t
       LEFT JOIN deadlines AS d
              ON t.labname = d.labs
WHERE  t.labname NOT IN ( "project1" ) ; ''', con = connection)

df_max

,uid,delta
0,user_25,2


In [16]:
df_avg = pd.io.sql.read_sql('''
SELECT Avg(Cast((Julianday(d.deadlines, 'unixepoch') -
                       Julianday(t.first_commit_ts) ) * 24
                           AS
                           INTEGER)) AS delta
FROM   test AS t
       LEFT JOIN deadlines AS d
              ON t.labname = d.labs
WHERE  t.labname NOT IN ( "project1" ) ; ''', con = connection)

df_avg

,delta
0,89.125


In [17]:
views_diff = pd.read_sql('''SELECT DISTINCT t.uid,
                                            t.avg_diff,
                                            p.pageviews
                            FROM 
                             (SELECT uid,
                                     AVG(CAST((julianday(d.deadlines, 'unixepoch') - julianday(test.first_commit_ts))*24 AS integer)) AS avg_diff
                              FROM test
                              LEFT JOIN deadlines AS d
                              ON test.labname = d.labs
                              WHERE test.labname NOT IN ("project1")
                              GROUP BY uid) AS t
                            LEFT JOIN 
                             (SELECT uid,
                                     COUNT(datetime) AS pageviews
                              FROM pageviews
                              GROUP BY uid) AS p
                              ON t.uid = p.uid ;''', con = connection)

views_diff

,uid,avg_diff,pageviews
0,user_1,64.400000,28
1,user_10,74.800000,89
2,user_14,159.000000,143
3,user_17,61.600000,47
4,user_18,5.666667,3
5,user_19,98.750000,16
6,user_21,95.500000,10
7,user_25,92.600000,179
8,user_28,86.400000,149
9,user_3,105.400000,317


In [18]:
views_diff[['avg_diff','pageviews']].corr()

,avg_diff,pageviews
avg_diff,1.000000,0.279736
pageviews,0.279736,1.000000


In [19]:
# Определить среднее значение дельты before за период до введения ленты новостей и второе значение after после введения ленты новостей по
# некоторым задачам (кроме project1) для всех пользователей (не по отдельности).

test_results = pd.read_sql(''' 
SELECT times,
       Avg(diff) AS avg_diff
FROM   (SELECT uid,
               labname,
               Avg(( deadlines - Strftime('%s', first_commit_ts) ) / 3600) AS diff,
               CASE
                 WHEN ( ( Strftime('%s', first_commit_ts) ) - Strftime('%s', first_view_ts) ) > 0 
                 THEN 'after'
                 ELSE 'before'
               END                                                         AS times
        FROM   test
               LEFT JOIN deadlines
                      ON labname = labs
        WHERE  labname NOT IN ("project1" )
        GROUP  BY uid, times)
WHERE  uid IN (SELECT uid
               FROM   (SELECT uid,
                              CASE
                                WHEN ( ( Strftime('%s', first_commit_ts) ) - Strftime('%s', first_view_ts) ) > 0 
                                THEN 'after'
                                ELSE 'before'
                              END AS times
                       FROM   test
                       WHERE  labname NOT IN ("project1")
                       GROUP  BY uid,
                                 times)
               GROUP  BY uid
               HAVING Count(uid) = 2)
GROUP  BY times ; ''', con = connection)

test_results

,times,avg_diff
0,after,99.523810
1,before,66.047619


In [20]:
control_results = pd.read_sql('''
SELECT times,
       Avg(diff) AS avg_diff
FROM   (SELECT uid,
               labname,
               Avg(( deadlines - Strftime('%s', first_commit_ts) ) / 3600) AS diff,
               CASE
                 WHEN ( ( Strftime('%s', first_commit_ts) ) - Strftime('%s', first_view_ts) ) > 0 
                 THEN 'after'
                 ELSE 'before'
               end                                                         AS times
        FROM   control
               LEFT JOIN deadlines
                      ON labname = labs
        WHERE  labname NOT IN ("project1")
        GROUP  BY uid,
                  times)
WHERE  uid IN (SELECT uid
               FROM   (SELECT uid,
                              CASE
                                WHEN ( ( Strftime('%s', first_commit_ts) ) - Strftime('%s', first_view_ts) ) > 0 
                                THEN 'after'
                                ELSE 'before'
                              end AS times
                       FROM   control
                       WHERE  labname NOT IN ("project1")
                       GROUP  BY uid,
                                 times)
               GROUP  BY uid
               HAVING Count(uid) = 2)
GROUP  BY times ;  ''', con = connection)

control_results

,times,avg_diff
0,after,99.322222
1,before,98.033333


In [21]:
# Закрыть соединение с базой данных
connection.close()